In [16]:
import pandas as pd 

In [17]:
path="/Users/guoyalong/Downloads/injuries_2010-2020.csv"

In [18]:
injuries_2010_2020=pd.read_csv(path)
injuries_2010_2020.head()

,Date,Team,Acquired,Relinquished,Notes
0,2010-10-03,Bulls,NaN,Carlos Boozer,fractured bone in right pinky finger (out inde...
1,2010-10-06,Pistons,NaN,Jonas Jerebko,torn right Achilles tendon (out indefinitely)
2,2010-10-06,Pistons,NaN,Terrico White,broken fifth metatarsal in right foot (out ind...
3,2010-10-08,Blazers,NaN,Jeff Ayres,torn ACL in right knee (out indefinitely)
4,2010-10-08,Nets,NaN,Troy Murphy,strained lower back (out indefinitely)


In [31]:
acquired_df=injuries_2010_2020.dropna(subset=["Acquired"])
acquired_df

,Date,Team,Acquired,Relinquished,Notes
53,2010-10-27,Heat,Jerry Stackhouse,NaN,activated from IL
81,2010-10-27,Rockets,Jermaine Taylor,NaN,activated from IL
101,2010-10-29,Cavaliers,Samardo Samuels,NaN,activated from IL
103,2010-10-29,Celtics,Luke Harangody,NaN,activated from IL
105,2010-10-29,Grizzlies,Marc Gasol,NaN,activated from IL
...,...,...,...,...,...
27091,2020-09-08,Bucks,Giannis Antetokounmpo,NaN,returned to lineup
27094,2020-09-12,Lakers,JaVale McGee,NaN,returned to lineup
27100,2020-09-30,Lakers,Dion Waiters,NaN,activated from IL
27103,2020-10-02,Heat,Chris Silva,NaN,activated from IL


In [32]:
relinquished_df=injuries_2010_2020.dropna(subset=["Relinquished"])
relinquished_df

,Date,Team,Acquired,Relinquished,Notes
0,2010-10-03,Bulls,NaN,Carlos Boozer,fractured bone in right pinky finger (out inde...
1,2010-10-06,Pistons,NaN,Jonas Jerebko,torn right Achilles tendon (out indefinitely)
2,2010-10-06,Pistons,NaN,Terrico White,broken fifth metatarsal in right foot (out ind...
3,2010-10-08,Blazers,NaN,Jeff Ayres,torn ACL in right knee (out indefinitely)
4,2010-10-08,Nets,NaN,Troy Murphy,strained lower back (out indefinitely)
...,...,...,...,...,...
27097,2020-09-22,Celtics,NaN,Romeo Langford,surgery on right wrist (out for season)
27098,2020-09-23,Heat,NaN,Gabe Vincent,sore right knee (DTD)
27099,2020-09-30,Heat,NaN,Bam Adebayo,strained left shoulder (DTD)
27101,2020-10-02,Heat,NaN,Bam Adebayo,strained neck (DTD)
